## Setup for Inpainting Eraser Models
---
In this notebook, you are going to run a defined setup process for our inpainting eraser solution. Due to the size of the models, some of the step may take some time to complete. The entire notebook should finish within 1 hour. At the end of the notebook run, we should have a instended inference container in Elastic Container Registry (ECR) ready to host our models using SageMaker Endpoint (MME).

SageMaker MME is a service provided by Amazon SageMaker that allows multiple machine learning models to be hosted on a single endpoint. This means that multiple models can be deployed and managed together, making it easier to scale and maintain machine learning applications. With a multi-model endpoint, different models can be selected based on specific needs, allowing for more flexibility and efficiency. It also enables different types of models to be combined, such as computer vision and natural language processing models, to create more comprehensive applications.

Here is a high level breakdown of the setup steps:

1. Downloading pre-trained models
2. Package conda environment for additional model dependencies
3. Extend SageMaker managed Triton container with model checkpoints and conda packs pre-loaded
4. Push the container to AWS Elastic Container Registry (ECR)

---

This notebook will locally build a custom docker image. **We recommend to use pytorch kernel on SageMaker Notebook Instance using `ml.g4dn.xlarge`**

In [1]:
!pip install -Uq sagemaker

### Setup

In [1]:
import sagemaker
import boto3

import tarfile
import os

sagemaker_session = sagemaker.Session(boto_session=boto3.Session())
region = sagemaker_session.boto_region_name
account = sagemaker_session.account_id()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


### Download Pre-trained Models

#### Download and Package SAM Checkpoint
**Apache-2.0 license**

In [3]:
model_file_name = "sam_vit_h_4b8939.pth"
download_path = f"https://huggingface.co/spaces/abhishek/StableSAM/resolve/main/{model_file_name}"

!wget $download_path

--2024-05-09 22:54:33--  https://huggingface.co/spaces/abhishek/StableSAM/resolve/main/sam_vit_h_4b8939.pth
Resolving huggingface.co (huggingface.co)... 18.154.227.67, 18.154.227.87, 18.154.227.69, ...
Connecting to huggingface.co (huggingface.co)|18.154.227.67|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/47/d3/47d331d77ce5639cc128df17410f4744b11342191e8442f5cde65f20735d01f9/a7bf3b02f3ebf1267aba913ff637d9a2d5c33d3173bb679e46d9f338c26f262e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sam_vit_h_4b8939.pth%3B+filename%3D%22sam_vit_h_4b8939.pth%22%3B&Expires=1715554473&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNTU1NDQ3M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy80Ny9kMy80N2QzMzFkNzdjZTU2MzljYzEyOGRmMTc0MTBmNDc0NGIxMTM0MjE5MWU4NDQyZjVjZGU2NWYyMDczNWQwMWY5L2E3YmYzYjAyZjNlYmYxMjY3YWJhOTEzZmY2MzdkOWEyZDVjMzNkMzE3M2JiNjc5ZTQ2ZDlmM

In [4]:
sd_tar = f"docker/{model_file_name}.tar.gz"

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

make_tarfile(sd_tar, model_file_name)

### Download and Package LaMa Checkpoint
**Apache-2.0 license**

In [5]:
!wget https://huggingface.co/smartywu/big-lama/resolve/main/big-lama.zip
!unzip big-lama.zip

--2024-05-09 22:57:25--  https://huggingface.co/smartywu/big-lama/resolve/main/big-lama.zip
Resolving huggingface.co (huggingface.co)... 18.154.227.87, 18.154.227.7, 18.154.227.69, ...
Connecting to huggingface.co (huggingface.co)|18.154.227.87|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/51/45/51456bfbae988f737d159c165ee79a3168fdd10d70db51da3e2dcc0cb29aa7a5/f1b358ca24093b93a106183b98a3dea6e8ed09f3b43ea7251eb2c81e7b4575f6?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27big-lama.zip%3B+filename%3D%22big-lama.zip%22%3B&response-content-type=application%2Fzip&Expires=1715554645&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNTU1NDY0NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy81MS80NS81MTQ1NmJmYmFlOTg4ZjczN2QxNTljMTY1ZWU3OWEzMTY4ZmRkMTBkNzBkYjUxZGEzZTJkY2MwY2IyOWFhN2E1L2YxYjM1OGNhMjQwOTNiOTNhMTA2MTgzYjk4YTNkZWE2ZThlZDA5ZjNiNDNlYTcyNT

In [6]:
lama_dir = "big-lama"
lama_tar = f"docker/{lama_dir}.tar.gz"

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

make_tarfile(lama_tar, lama_dir)

LaMa needs additional source code script. We will clone the repo into our `model_repo` folder

In [7]:
!cd model_repo/lama/1 && git clone https://github.com/advimman/lama.git

Cloning into 'lama'...
remote: Enumerating objects: 443, done.
remote: Counting objects: 100% (248/248), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 443 (delta 131), reused 101 (delta 101), pack-reused 195
Receiving objects: 100% (443/443), 6.53 MiB | 41.81 MiB/s, done.
Resolving deltas: 100% (172/172), done.


#### Downloading Images and Modules from Inpaint Anything
**Apache-2.0 license**

In [8]:
!cd statics && wget https://raw.githubusercontent.com/geekyutao/Inpaint-Anything/main/example/fill-anything/sample1.png
!cd statics && wget https://raw.githubusercontent.com/geekyutao/Inpaint-Anything/main/example/remove-anything/dog.jpg

--2024-05-09 23:01:05--  https://raw.githubusercontent.com/geekyutao/Inpaint-Anything/main/example/fill-anything/sample1.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2358157 (2.2M) [image/png]
Saving to: ‘sample1.png.1’

100%[======================================>] 2,358,157   --.-K/s   in 0.01s   

2024-05-09 23:01:05 (198 MB/s) - ‘sample1.png.1’ saved [2358157/2358157]

--2024-05-09 23:01:05--  https://raw.githubusercontent.com/geekyutao/Inpaint-Anything/main/example/remove-anything/dog.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response.

### Docker Environment Preparation
Because the volume size of container may be larger than the available size in root directory of the notebook instance, we need to put the directory of docker data into the ```/home/ec2-user/SageMaker/docker``` directory.

By default, the root directory of docker is set as ```/var/lib/docker/```. We need to change the directory of docker to ```/home/ec2-user/SageMaker/docker```.

In [9]:
!cat /etc/docker/daemon.json

{
    "runtimes": {
        "nvidia": {
            "path": "nvidia-container-runtime",
            "runtimeArgs": []
        }
    }
}


In [10]:
!bash ./prepare-docker.sh

Redirecting to /bin/systemctl stop docker.service
  docker.socket
cp: cannot stat ‘daemon.json’: No such file or directory
Redirecting to /bin/systemctl start docker.service


### Package Conda Environment for each model

SageMaker NVIDIA Triton container images does not contain all the libraries two run our SAM and LaMa models. However, Triton allows you to bring additional dependencies using conda pack. Let's run the two cells below to create a `xxx_env.tar.gz` environment package for each model.

In [11]:
!cd docker && bash sam_conda_dependencies.sh

Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 24.4.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.4.0



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/sam_env

  added / updated specs:
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libgcc-ng-13.2.0           |       h77fa898_7         758 KB  conda-forge
    libgomp-13.2.0             |       h77fa898_7         412 KB  conda-forge
    ncurses-6.5                |       h59595ed_0         867 KB  conda-forge
    openssl-3.3.0              |       hd590300_0         2.8 MB  conda-forge
    ---------------------------------------------------------

In [12]:
!cd docker && bash lama_conda_dependencies.sh

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 24.4.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.4.0



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/lama_env

  added / updated specs:
    - python=3.8


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge 
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu 
  bzip2              conda-forge/linux-64::bzip2-1.0.8-hd590300_5 
  ca-certificates    conda-forge/linux-64::ca-certificates-2024.2.2-hbcca054_0 
  ld_impl_linux-64   conda-forge/linux-64::ld_impl_linux-64-2.40-h55db66e_0 
  libffi             conda-forge/linux-64::libffi-3.4.2-h7f98852_5 
  libgcc-ng          conda-forge/linux-64::libgcc-ng-13.2.0-h77fa898_7 
  

### Extend SageMaker Managed Triton Container

When we host these models on SageMaker MME. When invoked, model files will be loaded from S3 onto the instance. Due the large size of our models and model packages (SAM: 2.4GB + conda pack: 2.52 GB, LaMa: 0.38 GB + conda pack: 3.35GB), we are going to pre-load these files into the container. This will reduce model loading time and improve user experience during cold start.

In [5]:
# account mapping for SageMaker Triton Image
account_id_map = {
    "us-east-1": "785573368785",
    "us-east-2": "007439368137",
    "us-west-1": "710691900526",
    "us-west-2": "301217895009",
    "eu-west-1": "802834080501",
    "eu-west-2": "205493899709",
    "eu-west-3": "254080097072",
    "eu-north-1": "601324751636",
    "eu-south-1": "966458181534",
    "eu-central-1": "746233611703",
    "ap-east-1": "110948597952",
    "ap-south-1": "763008648453",
    "ap-northeast-1": "941853720454",
    "ap-northeast-2": "151534178276",
    "ap-southeast-1": "324986816169",
    "ap-southeast-2": "355873309152",
    "cn-northwest-1": "474822919863",
    "cn-north-1": "472730292857",
    "sa-east-1": "756306329178",
    "ca-central-1": "464438896020",
    "me-south-1": "836785723513",
    "af-south-1": "774647643957",
}



region = boto3.Session().region_name
if region not in account_id_map.keys():
    raise ("UNSUPPORTED REGION")

base = "amazonaws.com.cn" if region.startswith("cn-") else "amazonaws.com"
mme_triton_image_uri = (
    "{account_id}.dkr.ecr.{region}.{base}/sagemaker-tritonserver:22.12-py3".format(
        account_id=account_id_map[region], region=region, base=base
    )
)
triton_account_id = account_id_map[region]
mme_triton_image_uri

'785573368785.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tritonserver:22.12-py3'

Preview the docker file

In [14]:
!cat docker/Dockerfile

ARG BASE_IMAGE

FROM $BASE_IMAGE

#Install any additional libraries
RUN echo "Adding conda package to Docker image"
RUN mkdir -p /home/condpackenv/
RUN mkdir -p /home/models/

# Copy conda env
COPY sam_env.tar.gz /home/condpackenv/sam_env.tar.gz
COPY lama_env.tar.gz /home/condpackenv/lama_env.tar.gz

COPY sam_vit_h_4b8939.pth.tar.gz /temp/
COPY big-lama.tar.gz /temp/

# Install tar
RUN apt-get update && apt-get install -y tar
# RUN apt-get update && apt-get install ffmpeg libsm6 libxext6  -y

# Untar the file
RUN tar -xzf /temp/sam_vit_h_4b8939.pth.tar.gz -C /home/models/
RUN tar -xzf /temp/big-lama.tar.gz -C /home/models/

RUN rm /temp/sam_vit_h_4b8939.pth.tar.gz
RUN rm /temp/big-lama.tar.gz

### Build & push the new image to ECR

In [2]:
# New container image name
new_image_name = 'sagemaker-tritonserver-sam-lama'

In [6]:
%%capture build_output
!cd docker && bash build_and_push.sh "$new_image_name" "latest" "$mme_triton_image_uri" "$region" "$account" "$triton_account_id"

In [7]:
if 'Error response from daemon' in str(build_output):
    print(build_output)
    raise SystemExit('\n\n!!There was an error with the container build!!')
else:
    extended_triton_image_uri = str(build_output).strip().split('\n')[-1]
    
print(f"New image URI from ECR: {extended_triton_image_uri}")

New image URI from ECR: 654405684375.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tritonserver-sam-lama:latest


In [8]:
%store extended_triton_image_uri

Stored 'extended_triton_image_uri' (str)
